In [1]:
import pandas as pd

books=pd.read_csv("books_categorised.csv")

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True, top_k=None)
classifier("I love this!")

In [ ]:
classifier(books["description"])

In [ ]:
predictions=classifier(books["description"][0].split("."))
#sentence wise analysis for finer classification of overall tone

In [ ]:
sorted(predictions[0], key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156372398138046},
 {'label': 'disgust', 'score': 0.002628477057442069},
 {'label': 'fear', 'score': 0.06816234439611435},
 {'label': 'joy', 'score': 0.047942597419023514},
 {'label': 'neutral', 'score': 0.14038586616516113},
 {'label': 'sadness', 'score': 0.002122163074091077},
 {'label': 'surprise', 'score': 0.7296022176742554}]

In [ ]:
import numpy as np

emotion_labels =["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn=[]
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores={label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions=sorted(prediction["score"], key=lambda x:x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [ ]:
from tqdm import tqdm

for i in tqdm(range(10)):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [ ]:
from tqdm import tqdm

emotion_labels =["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn=[]
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [ ]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn

In [ ]:
emotions_df.head()

In [ ]:
books=pd.merge(books, emotions_df, on="isbn13", how="left")

In [ ]:
books.to_csv("books_with_emotions.csv", index=False)